In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
embeddings = None

[nltk_data] Downloading package punkt to /home/jorge/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jorge/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jorge/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Spliteo para obtener vectores de train y test

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train,test = train_test_split(train_df,test_size=0.33,random_state = 17)
train.reset_index(inplace=True,drop=True)
test.reset_index(inplace=True,drop=True)
y_train = train['target'].values
y_test = test['target'].values

In [3]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [4]:
processed, X_train = prepare_for_cnn(train)

Percentage of words covered in the embeddings = 0.6691291143396536


In [13]:
processed

,invalid_location_character_count,location_is_place,mean_encode,keyword_length,text_length,location_length,stop_word_count,punctuation_count,hashtag_count,mention_count,...,text_embedding_290,text_embedding_291,text_embedding_292,text_embedding_293,text_embedding_294,text_embedding_295,text_embedding_296,text_embedding_297,text_embedding_298,text_embedding_299
0,0,0,0.332394,7,137,9,8,2,0,0,...,-0.095388,0.010420,-0.137559,0.057483,0.004149,-0.003357,0.010531,-0.027733,0.050691,-0.003874
1,0,0,0.280751,6,117,0,12,0,0,0,...,-0.004451,0.003784,-0.093940,-0.007401,-0.119637,-0.038094,-0.047624,-0.061011,0.035588,-0.002136
2,0,0,0.600000,8,80,0,10,2,0,0,...,0.055859,0.084729,-0.073218,0.051086,-0.057727,-0.052509,-0.060059,-0.065416,-0.017188,-0.016446
3,4,0,0.557243,6,127,25,5,15,0,2,...,-0.086775,0.041582,-0.071882,0.044573,-0.020787,-0.062112,0.016305,-0.109480,0.047154,-0.120989
4,0,0,0.219078,12,44,0,3,8,1,0,...,0.009979,0.008453,-0.133545,0.085571,-0.168243,-0.097534,-0.000244,-0.144165,0.015339,-0.027222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,0,1,0.638310,5,105,17,3,5,0,0,...,-0.043355,-0.133185,-0.119283,-0.045953,-0.127252,-0.017334,-0.079007,-0.119819,-0.020020,0.124390
5096,0,1,0.420423,11,137,11,2,11,0,0,...,-0.049957,0.040430,0.012134,-0.002661,-0.030566,0.025269,0.071167,-0.076440,0.014407,0.018445
5097,0,0,0.839437,6,105,0,2,5,0,0,...,0.009125,0.053214,-0.100740,0.015198,0.075342,-0.050513,-0.169012,-0.083276,0.094217,-0.002344
5098,0,0,0.653208,7,132,13,3,1,0,0,...,0.040901,-0.013988,-0.059555,0.041462,-0.039307,0.086788,-0.073860,-0.136898,0.056950,0.062004


In [14]:
def RNN():
    model = Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)])
    return model

In [15]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 128)               203264    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 211,585
Trainable params: 211,585
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train, y_train, epochs=100, shuffle=True, validation_split=0.2)

Epoch 1/100
128/128 [==============================] - 2s 17ms/step - loss: 0.6549 - accuracy: 0.5809 - val_loss: 0.6661 - val_accuracy: 0.5265
Epoch 2/100
128/128 [==============================] - 1s 10ms/step - loss: 0.6289 - accuracy: 0.5824 - val_loss: 0.6495 - val_accuracy: 0.5363
Epoch 3/100
128/128 [==============================] - 1s 10ms/step - loss: 0.6105 - accuracy: 0.6034 - val_loss: 0.6348 - val_accuracy: 0.5833
Epoch 4/100
128/128 [==============================] - 1s 10ms/step - loss: 0.5926 - accuracy: 0.6382 - val_loss: 0.6330 - val_accuracy: 0.5775
Epoch 5/100
128/128 [==============================] - 1s 10ms/step - loss: 0.5779 - accuracy: 0.6645 - val_loss: 0.6225 - val_accuracy: 0.5922
Epoch 6/100
128/128 [==============================] - 1s 10ms/step - loss: 0.5612 - accuracy: 0.6821 - val_loss: 0.5937 - val_accuracy: 0.6735
Epoch 7/100
128/128 [==============================] - 1s 9ms/step - loss: 0.5450 - accuracy: 0.7044 - val_loss: 0.5823 - val_accuracy: 

Epoch 58/100
128/128 [==============================] - 1s 5ms/step - loss: 0.2994 - accuracy: 0.8725 - val_loss: 0.5148 - val_accuracy: 0.7863
Epoch 59/100
128/128 [==============================] - 1s 5ms/step - loss: 0.2966 - accuracy: 0.8725 - val_loss: 0.5181 - val_accuracy: 0.7814
Epoch 60/100
128/128 [==============================] - 1s 5ms/step - loss: 0.2955 - accuracy: 0.8694 - val_loss: 0.5130 - val_accuracy: 0.7912
Epoch 61/100
128/128 [==============================] - 1s 5ms/step - loss: 0.2927 - accuracy: 0.8757 - val_loss: 0.5240 - val_accuracy: 0.7794
Epoch 62/100
128/128 [==============================] - 1s 5ms/step - loss: 0.2926 - accuracy: 0.8706 - val_loss: 0.5220 - val_accuracy: 0.7814
Epoch 63/100
128/128 [==============================] - 1s 5ms/step - loss: 0.2899 - accuracy: 0.8730 - val_loss: 0.5240 - val_accuracy: 0.7794
Epoch 64/100
128/128 [==============================] - 1s 5ms/step - loss: 0.2869 - accuracy: 0.8745 - val_loss: 0.5254 - val_accuracy:

In [17]:
processed, final_test = prepare_for_cnn(test)
prediction = model.predict_classes(final_test)

Percentage of words covered in the embeddings = 0.7248732284488363
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [18]:
accuracy_score(test['target'], prediction)

0.7982491046557899

In [19]:
f1_score(test['target'], prediction)

0.7530443253774962